In [1]:
import requests
import ast
import json
from time import sleep,time
import os
from tqdm import tqdm

In [2]:
def fastaTojson(filePath):
    with open(filePath) as f:
        lines = [line.strip() for line in f.readlines()]
    proteins=dict()
    for line in lines:
        if line.startswith(">"):
            Id=line.replace(">","")
            proteins[Id]=""
        else:
            proteins[Id]+=line
    proteins_json=json.dumps(proteins)
    return proteins_json

def getDisomineResults(data_json):
    # Post Request
    urlBase="http://bio2byte.com/disomine"
    extension="/api/"
    url="{}{}".format(urlBase,extension)
    response = requests.post(url, json=data_json)
    post_results = ast.literal_eval(response.text)
    
    # Get request
    location = post_results["Location"]
    url = "{}{}".format(urlBase,location)
    wait=1
    start=time()
    while requests.get(url).status_code!=200:
        resp = requests.get(url)
        sleep(1)
        wait+=1
        if wait%120==0:
            print(resp.text)
    response=requests.get(url)
    get_results = ast.literal_eval(response.text)["results"]
    return get_results

def writeDisomineResults(predictions,outputFile):
    with open(outputFile,"w") as f:
        for predSet in predictions:
            Id = predSet['proteinID']
            seq = predSet['sequence']
            disomine = predSet["disomine"]

            header="""
****************************************************
* disorder predictions
*                                                  *
* Generated by DisoMine                            *
* (http://dynamine.ibsquare.be)                    *
*                                                  *
* for {}
****************************************************\n""".format(Id)
            f.write(header)
            for i,res in enumerate(seq):
                pred=disomine[i]
                toWrite="{}\t{}\n".format(res,round(pred,3))
                f.write(toWrite)    

In [ ]:
DIR = "../2020-04-06.FindTwinsAnyEvidenceBacteria/4.6.CDHIT/"
with open("disomineDone.list") as F:
    DONE=[LINE.strip() for LINE in F.readlines()]
for FILE in tqdm([F for F in os.listdir(DIR) if F.endswith(".fasta") if F not in DONE]):
    try:
        ID=FILE.replace(".fasta","")
        FILE_PATH="{}/{}".format(DIR,FILE)

        DATA_JSON = fastaTojson(FILE_PATH)
        RESULTS = getDisomineResults(DATA_JSON)
        OUTPUT= "AnyEvidenceBacteria/{}/{}_disomine.pred".format(ID,ID)
        writeDisomineResults(RESULTS,OUTPUT)
        with open("disomineDone.list","a") as f:
            f.write(FILE+"\n")
    except:
        with open("disomineFailed.list","a") as f:
            f.write(FILE+"\n")

  0%|          | 0/381 [00:00<?, ?it/s]

{"id":10014,"request_text":"We are processing your request, there are still 0 processes in queue","location":"not available","status":202,"result_id":0}
{"id":10014,"request_text":"We are processing your request, there are still 0 processes in queue","location":"not available","status":202,"result_id":0}
{"id":10014,"request_text":"We are processing your request, there are still 0 processes in queue","location":"not available","status":202,"result_id":0}
{"id":10014,"request_text":"We are processing your request, there are still 0 processes in queue","location":"not available","status":202,"result_id":0}
{"id":10014,"request_text":"We are processing your request, there are still 0 processes in queue","location":"not available","status":202,"result_id":0}
{"id":10014,"request_text":"We are processing your request, there are still 0 processes in queue","location":"not available","status":202,"result_id":0}
{"id":10014,"request_text":"We are processing your request, there are still 0 proc

In [ ]:
DIR = "../2020-04-06.FindTwinsAnyEvidenceBacteria/4.6.CDHIT/"
PROTEINS=dict()
for FILE in [F for F in os.listdir(DIR) if F.endswith(".fasta")]:
    FILE_PATH="{}/{}".format(DIR,FILE)
    with open(FILE_PATH) as F:
        LINES=[LINE.strip() for LINE in F.readlines()]
    for LINE in LINES:
        if LINE.startswith(">"):
            ID=LINE.replace(">","")
            PROTEINS[ID]=""
        else:
            PROTEINS[ID]+=LINE
PROTEIN_JSON=json.dumps(PROTEINS)
RESULTS = getDisomineResults(PROTEIN_JSON)
writeDisomineResults(RESULTS,"disomine.pred")